# Data Scaling Tutorial

This notebook is meant to introduce some data scaling methods used in machine learning. Scaling features in ML is used for many reasons. Some techniques within ML critically depend on features of on a common scale, such as L1/L2 regularization or nearest-neighbors techniques. In Neural Networks, scaling allows the network to learn the relative contributions of each feature without being dominated by the scale of one feature or another.

*Note:* data can be transformed and inverse-transformed using the methods below, but exact results cannot be recovered due to approximation errors.

## Environment

In [ ]:
# Environment
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import sys
# Local modules
sys.path.append('..')
import reproducibility
from utils import hash2
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from moisture_rnn_pkl import pkl2train
from moisture_rnn import create_rnn_data2

## Setup & Data Read


In [ ]:
file='test_CA_202401.pkl'
train = pkl2train([file])

In [ ]:
import yaml

with open("../params.yaml") as file:
    params = yaml.safe_load(file)["rnn"]
params

In [ ]:
case = 'KRNC1_202401'
rnn_dat = create_rnn_data2(train[case], params)
X = rnn_dat["X_train"]

In [ ]:
X[0:5, :]

## Min-Max Scaler

Rescales data to a given range, (0, 1) by default in `sklearn`. If $x$ is a feature vector, we calculate the transformation $x'$ by:

$$
x' = \frac{x-\min\{x\}}{\max\{x\}-\min\{x\}}
$$

Notice that $x'=0$ if $x$ equals the minimum, and $x'=1$ if $x$ equals the maximum, as desired.

### Manual Calculation

In [ ]:
min = X.min(axis=0)
max = X.max(axis=0)
print(f"X column mins: {min}")
print(f"X column maxs: {max}")

In [ ]:
X_scaled = (X - min) / (max - min)

print(X_scaled[0:5, :])

The scaled data should have column mins & maxes approximately equal to 0 and 1, respectively.

In [ ]:
print(f"X-scaled column mins: {X_scaled.min(axis=0)}")
print(f"X-scaled column maxs: {X_scaled.max(axis=0)}")

### Using `sklearn`

In [ ]:
scaler = MinMaxScaler()
scaler.fit(X)
X_scaled2 = scaler.transform(X)

In [ ]:
print(X_scaled2[0:5, :])

### Compare Difference

The difference between the methods should be approximately zero, or close to machine-epsilon.

In [ ]:
np.max(np.abs(X_scaled - X_scaled2))

## Standard Scaler

Scale features to mean 0 and standard deviation 1, equivalent to z-scores. This method assumes features are approximately normally distributed and will lead to strange results if not. If $x$ is a feature vector of length $N$, we calculate the standard transformation $x'$ by:

$$
x' = \frac{x-\mu}{s}
$$



$$\text{Where},\quad \mu = \frac{1}{N}\sum_{i=1}^n x_i \quad \text{And,}\quad s = \sqrt{\sum_{i=1}^n\frac{(x_i-\mu)^2}{N}}$$

### Manual Calculation

In [ ]:
mu = X.mean(axis=0)
s = X.std(axis=0)
print(f"X column means: {mu}")
print(f"X column sds: {s}")

In [ ]:
X_scaled = (X - mu)/s
print(X_scaled[0:5, :])

The resulting scaled data should have column means approximately equal to zero and column standard deviations approximately equal to one.

In [ ]:
print(f"X-scaled column means: {X_scaled.mean(axis=0)}")
print(f"X-scaled column sds: {X_scaled.std(axis=0)}")

### Using `sklearn`

In [ ]:
scaler = StandardScaler()
scaler.fit(X)
X_scaled2 = scaler.transform(X)

In [ ]:
print(X_scaled2[0:5, :])

### Compare Difference

The difference between the methods should be approximately zero, or close to machine-epsilon.

In [ ]:
np.max(np.abs(X_scaled - X_scaled2))

## References

- `MinMaxScaler` from Scikitlearn: https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.MinMaxScaler.html

- `StandardScaler` from Scikitlearn: https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html